In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')  #natural language processing toolkit 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')  # open multilanguage wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from ast import literal_eval   #To get rid from punctuation
from zipfile import ZipFile

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...


In [4]:
#reading dataset
zf=ZipFile('./Hotel_Reviews.zip')
data=pd.read_csv(zf.open("Hotel_Reviews.csv"))

In [ ]:
#data.head()

In [ ]:
#data.tail()

In [ ]:
#data.shape

In [ ]:
#data.info()

In [ ]:
#print(data.Hotel_Address.head())

In [5]:
#for simplicity, I will change countries to countries Alpha-2 codes
data.Hotel_Address=data.Hotel_Address.str.replace("Netherlands","NL")
data.Hotel_Address=data.Hotel_Address.str.replace("United Kingdom","UK")
data.Hotel_Address=data.Hotel_Address.str.replace("France","FR")
data.Hotel_Address=data.Hotel_Address.str.replace("Spain","ES")
data.Hotel_Address=data.Hotel_Address.str.replace("Italy","IT")
data.Hotel_Address=data.Hotel_Address.str.replace("Austria","AT")

In [6]:
#splitting hotel address and picking out last string which would be the countries
# and store them in new column by the name
data['countries']=data.Hotel_Address.apply(lambda x: x.split(' ')[-1])

In [ ]:
#print(data.countries.unique())

In [ ]:
#data.columns

In [ ]:
"""dropping unnecessary columns
As I need to reduce the size of dataset for uploading it to github so I did that manually but if you are downloading
dataset from the official kaggel website so you have to run this column 
data.drop(['Additional_Number_of_Scoring','Review_Date','Reviewer_Nationality',
          'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review','Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
          'days_since_review', 'lat', 'lng'],1,inplace=True)"""

"dropping unnecessary columns\nAs I need to reduce the size of dataset for uploading it to github so I did that manually but if you are downloading\ndataset from the official kaggel website so you have to run this column \ndata.drop(['Additional_Number_of_Scoring','Review_Date','Reviewer_Nationality',\n          'Negative_Review', 'Review_Total_Negative_Word_Counts',\n       'Total_Number_of_Reviews', 'Positive_Review','Review_Total_Positive_Word_Counts',\n       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',\n          'days_since_review', 'lat', 'lng'],1,inplace=True)"

In [ ]:
#data.columns

In [7]:
# creating a function to convert the string of list into a normal list
# getting rid of punctuation
def impute(column):
    column=column[0]
    if(type(column)!=list):
        return "".join(literal_eval(column))
    else:
        return column

In [ ]:
#check on tags column
#data.head()

In [8]:
# applying impute function to the "Tags" column in dataset
# To get rid of punctuation(commas and apostrophes)
data['Tags']=data[["Tags"]].apply(impute,axis=1)
#data.head()

C:\Users\ahmed\AppData\Local\Temp\ipykernel_14264\3092971875.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  column=column[0]


In [9]:
#for simplicity, I will convert "tags" and "countries" columns to lowercase
data['Tags']=data['Tags'].str.lower()
data['countries']=data['countries'].str.lower()

In [10]:
# defining recommender funtion
def recommender(location,description):
    #dividing the texts into small tokens(sentences into words)
    description=description.lower()
    word_tokenize(description)
    #applying stopwords
    stop_words=stopwords.words('english')
    #apply lemmatization
    lemm=WordNetLemmatizer()
    #connecting packages and filtering
    filtered=(word for word in description if not word in stop_words)
    filtered_set=set()
    for fs in filtered:
        filtered_set.add(lemm.lemmatize(fs))
# creating a variable that takes in location and returns following features
        country=data[data['countries']==location.lower()]
        country=country.set_index(np.arange(country.shape[0]))
        list1=[]; list2=[]; cos=[];
        for i in range(country.shape[0]):
            temp_token=word_tokenize(country['Tags'][i])
            temp_set=[word for word in temp_token if not word in stop_words]
            temp2_set=set()
            for s in temp_set:
                temp2_set.add(lemm.lemmatize(s))
            vector =temp2_set.intersection(filtered_set)
            cos.append(len(vector))
            # applying cosine similarity
        country['similarity']=cos
        country=country.sort_values(by='similarity',ascending=False)
        country.drop_duplicates(subset='Hotel_Name',keep='first',inplace=True)
        country.sort_values('Average_Score',ascending=False,inplace=True)
        country.reset_index(inplace=True)
        return country[['Hotel_Name','Average_Score','Hotel_Address']].head(10)
        